In [ ]:

# # Foreacasting with SARIMA
#
# In depth evaluation and testing of models supplemented by graphs, plots and tables of MAPE for different time steps ahead in terms of prediction.
#
# The problem framing:
# forecast periods p = {6, 12, 18, 24, 36} hours
#
# - predict the period:
#     - single step of p
#     - multistep the consecutive p step values
#


In [ ]:


get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [ ]:


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:


from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:


from utils import *

In [ ]:


plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:





# ### Load preprocessed data into dataframe

In [ ]:


filename = 'processed_series.csv'
df = pd.read_csv(filename,
                 low_memory=False,
                 index_col='Date', parse_dates=True
                )
df.columns = ['Value']

print(df.shape)
df.head()


# ### Resample to 6 hours

In [ ]:


df_six_hr = df.Value.resample('6H').mean().to_frame()

In [ ]:


print('DF shape:', df_six_hr.shape)
df_six_hr.head()

In [ ]:


df_six_hr.describe()


# ### Transformations
#
# Scale using StandardScaler

In [ ]:


scaler = StandardScaler()
scaled = scaler.fit_transform(df_six_hr)

In [ ]:


df_scaled = df_six_hr.copy()
df_scaled[:] = scaled
df_scaled.head()

In [ ]:


df_scaled.describe()

In [ ]:





# ### Split into train and test sets
#
# Using the first 40 years for training, and remaining 5 years for testing.

In [ ]:


split_year = '2014'

s_train = df_scaled[:split_year].values[:, 0]
s_test = df_scaled[split_year:].values[:, 0]

print('s_train shape', s_train.shape)
print('s_test shape', s_test.shape)


# ### Autocorelations
#
# - ACF
# - PACF

In [ ]:


n_lags = 365 * 4 # 6 hours period
lags = np.arange(0, n_lags, 20)
limit = -1

In [ ]:


fig, ax = plt.subplots(3, figsize=(12, 6))

plot_acf(s_train[:limit], ax=ax[0], lags=lags)
plot_pacf(s_train[:limit], ax=ax[1], lags=lags)
ax[2].plot(s_train[:limit])

plt.tight_layout()

In [ ]:


n_lags


# ### Fit SARIMA on training set
#
# - Non-seasonal: (d, p, q) = (0, 1, 2)
#
# - Seasonal: (D, P, Q) M = (0, 1, 2) lags

In [ ]:


sarima_model = SARIMAX(s_train, order=(0, 1, 2), seasonal_order=(0, 1, 2, n_lags),
                       enforce_invertibility=False, enforce_stationarity=False)
sarima_fit = sarima_model.fit()

In [ ]:


sarima_fit.summary()

In [ ]:


# Forecast period
start = s_train.shape[0]
end = start + s_test.shape[0]
start, end

In [ ]:


p = sarima_fit.get_prediction(start, end)

In [ ]:


predicted_means = p.predicted_mean
predicted_intervals = p.conf_int(alpha=0.05)

In [ ]:


predicted_means.shape, s_test.shape

In [ ]:


lower_bounds = predicted_intervals[:, 0]
upper_bounds = predicted_intervals[:, 1]

In [ ]:


lower_bounds.shape

In [ ]:


n_steps = 6

In [ ]:


def recursive_forecast(p_means, p_intervals, y_test, n_steps=1):
    # Make an accumulator for predictions
    predictions = np.zeros(shape=(y_test.shape[0], n_steps))
    predictions[:] = np.nan

    for i in range(n_steps):
        predictions[:, i] = p_means[1:]

    return predictions

In [ ]:


# Make recursive multi-step predictions
y_pred_multi = recursive_forecast(predicted_means, predicted_intervals, s_test, n_steps)

# Evaluate
svr_rmse = eval_multi(s_test, y_pred_multi, calc_rmse, scaler)
svr_mape = eval_multi(s_test, y_pred_multi, calc_mape, scaler)

# Report the metrics
metrics = np.array([svr_rmse, svr_mape]).T
summary = report_metrics(metrics, ['RMSE', 'MAPE'])

In [ ]:


summary

In [ ]:


visualize_pred(s_test, y_pred_multi, 'SARIMA Model',
               df_scaled, split_year, scaler)

In [ ]:


sarima_rmse = calc_rmse(s_test, p.predicted_mean[:-1])
print('SARIMA RMSE:', sarima_rmse)

In [ ]:


fig, ax = plt.subplots(figsize=(20, 10))

# # Training set
# ax.plot(df_scaled[:split_year].index, s_train, label='Train')

# Test set truth
ax.plot(df_scaled[split_year:].index, s_test, label='Test truth')

# Test prediction
ax.plot(df_scaled[split_year:].index, predicted_means[:-1],
        color='#ff7823', linestyle='--',
        label="prediction (RMSE={:0.4f})".format(sarima_rmse) )

# Prediction boundaries
ax.fill_between(df_scaled[split_year:].index, lower_bounds[:-1], upper_bounds[:-1],
                color='#ff7823', alpha=0.3,
                label="Confidence interval (95%)")

ax.legend();
ax.set_title("SARIMA Model");

In [ ]:





# ### Results summary
# MAPE values

In [ ]:


results = [summary]
names = ['SARIMA']

mape_results = [res.MAPE for res in results]

In [ ]:


_summary = pd.concat(mape_results, axis=1)
_summary.columns = names

In [ ]:


_summary.T